(sphx_glr_tutorial_tvmc_command_line_driver.py)=
# 用 TVMC 编译和优化模型

原作者：[Leandro Nunes](https://github.com/leandron), [Matthew Barrett](https://github.com/mbaret), [Chris Hoge](https://github.com/hogepodge)

在本节中，将使用 TVMC，即 TVM 命令行驱动程序。TVMC 工具，它暴露了 TVM 的功能，如 auto-tuning、编译、profiling 和通过命令行界面执行模型。

在完成本节内容后，将使用 TVMC 来完成以下任务：

* 为 TVM 运行时编译预训练 ResNet-50 v2 模型。
* 通过编译后的模型运行真实图像，并解释输出和模型的性能。
* 使用 TVM 在 CPU 上调优模型。
* 使用 TVM 收集的调优数据重新编译优化模型。
* 通过优化后的模型运行图像，并比较输出和模型的性能。

本节的目的是让你了解 TVM 和 TVMC 的能力，并为理解 TVM 的工作原理奠定基础。

## 使用 TVMC

TVMC 是 Python 应用程序，是 TVM Python 软件包的一部分。当你使用 Python 包安装 TVM 时，你将得到 TVMC 作为命令行应用程序，名为 ``tvmc``。这个命令的位置将取决于你的平台和安装方法。

另外，如果你在 ``$PYTHONPATH`` 上将 TVM 作为 Python 模块，你可以通过可执行的 python 模块 ``python -m tvm.driver.tvmc`` 访问命令行驱动功能。

为简单起见，本教程将提到 TVMC 命令行使用 ``tvmc <options>``，但同样的结果可以用 ``python -m tvm.driver.tvmc <options>``。

你可以使用帮助页面查看：

In [2]:
!python -m tvm.driver.tvmc --help

usage: tvmc [--config CONFIG] [-v] [--version] [-h]
            {micro,run,tune,compile} ...

TVM compiler driver

options:
  --config CONFIG       configuration json file
  -v, --verbose         increase verbosity
  --version             print the version and exit
  -h, --help            show this help message and exit.

commands:
  {micro,run,tune,compile}
    micro               select micro context.
    run                 run a compiled module
    tune                auto-tune a model
    compile             compile a model.

TVMC - TVM driver command-line interface


``tvmc`` 可用的 TVM 的主要功能来自子命令 ``compile`` 和 ``run``，以及 ``tune``。要了解某个子命令下的具体选项，请使用 ``tvmc <subcommand> --help``。将在本教程中逐一介绍这些命令，但首先需要下载预训练模型来使用。

## 获得模型

在本教程中，将使用 ResNet-50 v2。ResNet-50 是卷积神经网络，有 50 层深度，设计用于图像分类。将使用的模型已经在超过一百万张图片上进行了预训练，有 1000 种不同的分类。该网络输入图像大小为 224x224。如果你有兴趣探究更多关于 ResNet-50 模型的结构，建议下载 `[Netron](https://netron.app)，它免费提供的 ML 模型查看器。

在本教程中，将使用 ONNX 格式的模型。

In [4]:
!wget https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v2-7.onnx

--2022-04-26 13:07:52--  https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/onnx/models/main/vision/classification/resnet/model/resnet50-v2-7.onnx [following]
--2022-04-26 13:07:53--  https://media.githubusercontent.com/media/onnx/models/main/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102442450 (98M) [application/octet-stream]
Saving to: ‘resnet50-v2-7.onnx’

resnet50-v2-7.onnx  100%[===================>]  97.70M  4.51MB/s    in

为了让该模型可以被其他教程使用，需要：

In [5]:
!mv resnet50-v2-7.onnx ../../_models/resnet50-v2-7.onnx

```{admonition} 支持的模型格式
TVMC 支持用 Keras、ONNX、TensorFlow、TFLite 和 Torch 创建的模型。如果你需要明确地提供你所使用的模型格式，请使用选项 ``--model-format``。
```

更多信息见：

In [6]:
!python -m tvm.driver.tvmc compile --help

usage: tvmc compile [-h] [--cross-compiler CROSS_COMPILER]
                    [--cross-compiler-options CROSS_COMPILER_OPTIONS]
                    [--desired-layout {NCHW,NHWC}] [--dump-code FORMAT]
                    [--model-format {keras,onnx,pb,tflite,pytorch,paddle}]
                    [-o OUTPUT] [-f {so,mlf}] [--pass-config name=value]
                    [--target TARGET]
                    [--target-example_target_hook-from_device TARGET_EXAMPLE_TARGET_HOOK_FROM_DEVICE]
                    [--target-example_target_hook-libs TARGET_EXAMPLE_TARGET_HOOK_LIBS]
                    [--target-example_target_hook-model TARGET_EXAMPLE_TARGET_HOOK_MODEL]
                    [--target-example_target_hook-tag TARGET_EXAMPLE_TARGET_HOOK_TAG]
                    [--target-example_target_hook-device TARGET_EXAMPLE_TARGET_HOOK_DEVICE]
                    [--target-example_target_hook-keys TARGET_EXAMPLE_TARGET_HOOK_KEYS]
                    [--target-ext_dev-from_device TARGET_EXT_DEV_FR

```{admonition} 为 TVM 添加 ONNX 支持
TVM 依赖于你系统中的 ONNX python 库。你可以使用 ``pip3 install --user onnx onnxoptimizer`` 命令来安装 ONNX。如果你有 root 权限并且想全局安装 ONNX，你可以去掉 ``--user`` 选项。对 ``onnxoptimizer`` 的依赖是可选的，仅用于 ``onnx>=1.9``。
```

## 将 ONNX 模型编译到 TVM 运行时中

一旦下载了 ResNet-50 模型，下一步就是对其进行编译。为了达到这个目的，将使用 ``tvmc compile``。从编译过程中得到的输出是模型的 TAR 包，它被编译成目标平台的动态库。可以使用 TVM 运行时在目标设备上运行该模型。

In [5]:
# 这可能需要几分钟的时间，取决于你的机器
!python -m tvm.driver.tvmc compile --target "llvm" \
    --output resnet50-v2-7-tvm.tar \
        ../../_models/resnet50-v2-7.onnx

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


查看 ``tvmc compile`` 在 module 中创建的文件：

In [7]:
%%bash
mkdir model
tar -xvf resnet50-v2-7-tvm.tar -C model

mod.so
mod.json
mod.params


列出了三个文件：

* ``mod.so`` 是模型，表示为 C++ 库，可以被 TVM 运行时加载。
* ``mod.json`` 是 TVM Relay 计算图的文本表示。
* ``mod.params`` 是包含预训练模型参数的文件。

该 module 可以被你的应用程序直接加载，而 model 可以通过 TVM 运行时 API 运行。

```{admonition} 定义正确的 target
指定正确的目标（选项 ``--target``）可以对编译后的模块的性能产生巨大的影响，因为它可以利用目标上可用的硬件特性。
  
欲了解更多信息，请参考 [为 x86 CPU 自动调优卷积网络](tune_relay_x86)。建议确定你运行的是哪种 CPU，以及可选的功能，并适当地设置目标。
```

## 用 TVMC 从编译的模块中运行模型

已经将模型编译到模块，可以使用 TVM 运行时来进行预测。


TVMC 内置了 TVM 运行时，允许你运行编译的 TVM 模型。为了使用 TVMC 来运行模型并进行预测，需要两样东西：

- 编译后的模块，我们刚刚生成出来。
- 对模型的有效输入，以进行预测。

当涉及到预期的张量形状、格式和数据类型时，每个模型都很特别。出于这个原因，大多数模型需要一些预处理和后处理，以确保输入是有效的，并解释输出结果。TVMC 对输入和输出数据都采用了 NumPy 的 ``.npz`` 格式。这是得到良好支持的 NumPy 格式，可以将多个数组序列化为文件。

作为本教程的输入，将使用一只猫的图像，但你可以自由地用你选择的任何图像来代替这个图像。

### 输入预处理

对于 ResNet-50 v2 模型，预期输入是 ImageNet 格式的。下面是为 ResNet-50 v2 预处理图像的脚本例子。

你将需要安装支持的 Python 图像库的版本。你可以使用 ``pip3 install --user pillow`` 来满足脚本的这个要求。

In [8]:
#!python ./preprocess.py
from tvm.contrib.download import download_testdata
from PIL import Image
import numpy as np

img_url = "https://s3.amazonaws.com/model-server/inputs/kitten.jpg"
img_path = download_testdata(img_url, "imagenet_cat.png", module="data")

# Resize it to 224x224
resized_image = Image.open(img_path).resize((224, 224))
img_data = np.asarray(resized_image).astype("float32")

# ONNX expects NCHW input, so convert the array
img_data = np.transpose(img_data, (2, 0, 1))

# Normalize according to ImageNet
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_stddev = np.array([0.229, 0.224, 0.225])
norm_img_data = np.zeros(img_data.shape).astype("float32")
for i in range(img_data.shape[0]):
      norm_img_data[i, :, :] = (img_data[i, :, :] / 255 - imagenet_mean[i]) / imagenet_stddev[i]

# Add batch dimension
img_data = np.expand_dims(norm_img_data, axis=0)

# Save to .npz (outputs imagenet_cat.npz)
np.savez("imagenet_cat", data=img_data)

### 运行已编译的模块

有了模型和输入数据，现在可以运行 TVMC 来做预测：

In [10]:
!python -m tvm.driver.tvmc run \
    --inputs imagenet_cat.npz \
    --output predictions.npz \
    resnet50-v2-7-tvm.tar

回顾一下， ``.tar`` 模型文件包括 C++ 库，对 Relay 模型的描述，以及模型的参数。TVMC 包括 TVM 运行时，它可以加载模型并根据输入进行预测。当运行上述命令时，TVMC 会输出新文件，``predictions.npz``，其中包含 NumPy 格式的模型输出张量。

在这个例子中，在用于编译的同一台机器上运行该模型。在某些情况下，可能想通过 RPC Tracker 远程运行它。要阅读更多关于这些选项的信息，请查看：

In [11]:
!python -m tvm.driver.tvmc run --help

usage: tvmc run [-h] [--device {cpu,cuda,cl,metal,vulkan,rocm,micro}]
                [--fill-mode {zeros,ones,random}] [-i INPUTS] [-o OUTPUTS]
                [--print-time] [--print-top N] [--profile] [--end-to-end]
                [--repeat N] [--number N] [--rpc-key RPC_KEY]
                [--rpc-tracker RPC_TRACKER] [--list-options]
                PATH

positional arguments:
  PATH                  path to the compiled module file or to the project
                        directory if '--device micro' is selected.

optional arguments:
  -h, --help            show this help message and exit
  --device {cpu,cuda,cl,metal,vulkan,rocm,micro}
                        target device to run the compiled module. Defaults to
                        'cpu'
  --fill-mode {zeros,ones,random}
                        fill all input tensors with values. In case
                        --inputs/-i is provided, they will take precedence
                        over --fill-mode. Any remaining input

### 输出后处理

如前所述，每个模型都会有自己的特定方式来提供输出张量。

需要运行一些后处理，利用为模型提供的查找表，将 ResNet-50 v2 的输出渲染成人类可读的形式。

下面的脚本显示了后处理的例子，从编译的模块的输出中提取标签。

运行这个脚本应该产生以下输出：

In [12]:
#!python ./postprocess.py
import os.path
import numpy as np

from scipy.special import softmax

from tvm.contrib.download import download_testdata

# Download a list of labels
labels_url = "https://s3.amazonaws.com/onnx-model-zoo/synset.txt"
labels_path = download_testdata(labels_url, "synset.txt", module="data")

with open(labels_path, "r") as f:
    labels = [l.rstrip() for l in f]

output_file = "predictions.npz"

# Open the output and read the output tensor
if os.path.exists(output_file):
    with np.load(output_file) as data:
        scores = softmax(data["output_0"])
        scores = np.squeeze(scores)
        ranks = np.argsort(scores)[::-1]

        for rank in ranks[0:5]:
            print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

class='n02123045 tabby, tabby cat' with probability=0.621104
class='n02123159 tiger cat' with probability=0.356378
class='n02124075 Egyptian cat' with probability=0.019712
class='n02129604 tiger, Panthera tigris' with probability=0.001215
class='n04040759 radiator' with probability=0.000262


试着用其他图像替换猫的图像，看看 ResNet 模型会做出什么样的预测。

## 自动调优 ResNet 模型

之前的模型是为了在 TVM 运行时工作而编译的，但不包括任何特定平台的优化。在本节中，将展示如何使用 TVMC 建立针对你工作平台的优化模型。

在某些情况下，当使用编译模块运行推理时，可能无法获得预期的性能。在这种情况下，可以利用自动调优器，为模型找到更好的配置，获得性能的提升。TVM 中的调优是指对模型进行优化以在给定目标上更快地运行的过程。这与训练或微调不同，因为它不影响模型的准确性，而只影响运行时的性能。作为调优过程的一部分，TVM 将尝试运行许多不同的运算器实现变体，以观察哪些算子表现最佳。这些运行的结果被存储在调优记录文件中，这最终是 ``tune`` 子命令的输出。

在最简单的形式下，调优要求你提供三样东西：

- 你打算在这个模型上运行的设备的目标规格
- 输出文件的路径，调优记录将被保存在该文件中
- 最后是要调优的模型的路径。

默认搜索算法需要 `xgboost`，请参阅下面关于优化搜索算法的详细信息：

```bash
pip install xgboost cloudpickle
```

下面的例子展示了这一做法的实际效果：

In [1]:
!python -m tvm.driver.tvmc tune --target "llvm" \
    --output resnet50-v2-7-autotuner_records.json \
        ../../_models/resnet50-v2-7.onnx

/media/pc/data/4tb/lxw/anaconda3/envs/mx39/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[Task  1/25]  Current/Best:  139.87/ 252.51 GFLOPS | Progress: (40/40) | 20.88 s Done.
[Task  2/25]  Current/Best:   42.44/ 183.76 GFLOPS | Progress: (40/40) | 11.12 s Done.
[Task  3/25]  Current/Best:  176.21/ 215.65 GFLOPS | Progress: (40/40) | 11.55 s Done.
[Task  4/25]  Current/Best:  113.94/ 160.83 GFLOPS | Progress: (40/40) | 13.36 s Done.
[Task  5/25]  Current/Best:  120.38/ 164.05 GFLOPS | Progress: (40/40) | 12.15 s Done.
[Task  6/25]  Current/Best:  103.44/ 188.69 GFLOPS | Progress: (40/40) | 12.60 s Done.
[Task  7/25]  Current/Best:  137.09/ 204.00 GFLOPS | Progress: (40/40) | 11.36 s Done.
[Task  8/25]  Current/Best:   99.24/ 195.34 GFLOPS | Progress: (40/40) | 18.87 s Done.
[Task  9/25]  

在这个例子中，如果你为 ``--target`` 标志指出更具体的目标，你会看到更好的结果。

TVMC 将对模型的参数空间进行搜索，尝试不同的运算符配置，并选择在你的平台上运行最快的一个。尽管这是基于 CPU 和模型操作的指导性搜索，但仍可能需要几个小时来完成搜索。这个搜索的输出将被保存到 ``resnet50-v2-7-autotuner_records.json`` 文件中，以后将被用来编译优化的模型。

```{admonition} 定义调优搜索算法
默认情况下，这种搜索是使用 ``XGBoost Grid`` 算法引导的。根据你的模型的复杂性和可利用的时间，你可能想选择不同的算法。完整的列表可以通过查阅：
```

In [2]:
!python -m tvm.driver.tvmc tune --help

usage: tvmc tune [-h] [--early-stopping EARLY_STOPPING]
                 [--min-repeat-ms MIN_REPEAT_MS]
                 [--model-format {keras,onnx,pb,tflite,pytorch,paddle}]
                 [--number NUMBER] -o OUTPUT [--parallel PARALLEL]
                 [--repeat REPEAT] [--rpc-key RPC_KEY]
                 [--rpc-tracker RPC_TRACKER] [--target TARGET]
                 [--target-example_target_hook-from_device TARGET_EXAMPLE_TARGET_HOOK_FROM_DEVICE]
                 [--target-example_target_hook-libs TARGET_EXAMPLE_TARGET_HOOK_LIBS]
                 [--target-example_target_hook-model TARGET_EXAMPLE_TARGET_HOOK_MODEL]
                 [--target-example_target_hook-tag TARGET_EXAMPLE_TARGET_HOOK_TAG]
                 [--target-example_target_hook-device TARGET_EXAMPLE_TARGET_HOOK_DEVICE]
                 [--target-example_target_hook-keys TARGET_EXAMPLE_TARGET_HOOK_KEYS]
                 [--target-ext_dev-from_device TARGET_EXT_DEV_FROM_DEVICE]
                 [--target-ext_dev-

对于消费级 Skylake CPU 来说，输出结果将是这样的：

In [3]:
!python -m tvm.driver.tvmc tune \
    --target "llvm -mcpu=broadwell" \
        --output resnet50-v2-7-autotuner_records.json \
            ../../_models/resnet50-v2-7.onnx

/media/pc/data/4tb/lxw/anaconda3/envs/mx39/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[Task  1/25]  Current/Best:  135.54/ 444.49 GFLOPS | Progress: (40/40) | 16.09 s Done.
[Task  2/25]  Current/Best:   91.39/ 426.70 GFLOPS | Progress: (40/40) | 10.33 s Done.
[Task  3/25]  Current/Best:  147.25/ 516.21 GFLOPS | Progress: (40/40) | 11.55 s Done.
[Task  4/25]  Current/Best:  561.81/ 561.81 GFLOPS | Progress: (40/40) | 12.99 s Done.
[Task  5/25]  Current/Best:  182.70/ 570.25 GFLOPS | Progress: (40/40) | 11.12 s Done.
[Task  6/25]  Current/Best:   79.82/ 459.29 GFLOPS | Progress: (40/40) | 12.03 s Done.
[Task  7/25]  Current/Best:  152.79/ 300.64 GFLOPS | Progress: (40/40) | 11.16 s Done.
[Task  8/25]  Current/Best:  155.29/ 310.77 GFLOPS | Progress: (40/40) | 14.68 s Done.
[Task  9/25]  

调谐会话可能需要很长的时间，所以 ``tvmc tune`` 提供了许多选项来定制你的调谐过程，在重复次数方面（例如 ``--repeat`` 和 ``--number``），要使用的调谐算法等等。

## 用调优数据编译优化后的模型

作为上述调谐过程的输出，获得了存储在 ``resnet50-v2-7-autotuner_records.json`` 的调谐记录。这个文件可以有两种使用方式：

- 作为进一步调谐的输入（通过 ``tvmc tune --tuning-records``）。
- 作为对编译器的输入

编译器将使用这些结果来为你指定的目标上的模型生成高性能代码。要做到这一点，可以使用 ``tvmc compile --tuning-records``。

获得更多信息：

In [4]:
!python -m tvm.driver.tvmc compile --help

usage: tvmc compile [-h] [--cross-compiler CROSS_COMPILER]
                    [--cross-compiler-options CROSS_COMPILER_OPTIONS]
                    [--desired-layout {NCHW,NHWC}] [--dump-code FORMAT]
                    [--model-format {keras,onnx,pb,tflite,pytorch,paddle}]
                    [-o OUTPUT] [-f {so,mlf}] [--pass-config name=value]
                    [--target TARGET]
                    [--target-example_target_hook-from_device TARGET_EXAMPLE_TARGET_HOOK_FROM_DEVICE]
                    [--target-example_target_hook-libs TARGET_EXAMPLE_TARGET_HOOK_LIBS]
                    [--target-example_target_hook-model TARGET_EXAMPLE_TARGET_HOOK_MODEL]
                    [--target-example_target_hook-tag TARGET_EXAMPLE_TARGET_HOOK_TAG]
                    [--target-example_target_hook-device TARGET_EXAMPLE_TARGET_HOOK_DEVICE]
                    [--target-example_target_hook-keys TARGET_EXAMPLE_TARGET_HOOK_KEYS]
                    [--target-ext_dev-from_device TARGET_EXT_DEV_FR

现在，模型的调谐数据已经收集完毕，可以使用优化的算子重新编译模型，以加快计算速度。

In [5]:
!python -m tvm.driver.tvmc compile \
    --target "llvm" \
        --tuning-records resnet50-v2-7-autotuner_records.json  \
            --output resnet50-v2-7-tvm_autotuned.tar \
                ../../_models/resnet50-v2-7.onnx

验证优化后的模型是否运行并产生相同的结果：

In [7]:
!python -m tvm.driver.tvmc run \
    --inputs imagenet_cat.npz \
        --output predictions.npz \
            resnet50-v2-7-tvm_autotuned.tar

!python postprocess.py

class='n02123045 tabby, tabby cat' with probability=0.621104
class='n02123159 tiger cat' with probability=0.356378
class='n02124075 Egyptian cat' with probability=0.019712
class='n02129604 tiger, Panthera tigris' with probability=0.001215
class='n04040759 radiator' with probability=0.000262


## 比较已调谐和未调谐的模型

TVMC 提供了在模型之间进行基本性能基准测试的工具。你可以指定重复次数，并且 TVMC 报告模型的运行时间（与运行时间的启动无关）。可以粗略了解调谐对模型性能的改善程度。例如，在测试的英特尔 i7 系统上，看到调谐后的模型比未调谐的模型运行快  $47\%$。

In [8]:
!python -m tvm.driver.tvmc run \
    --inputs imagenet_cat.npz \
        --output predictions.npz  \
            --print-time \
                --repeat 100 \
                    resnet50-v2-7-tvm_autotuned.tar

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  41.2506      40.8879      54.4469      36.7249       2.4430   
               


In [9]:
!python -m tvm.driver.tvmc run \
    --inputs imagenet_cat.npz \
        --output predictions.npz  \
            --print-time \
                --repeat 100 \
                    resnet50-v2-7-tvm.tar

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  51.8327      52.5906      67.5374      42.9440       4.4040   
               


## 小结

在本教程中，介绍了 TVMC，用于 TVM 的命令行驱动。演示了如何编译、运行和调优模型。还讨论了对输入和输出进行预处理和后处理的必要性。在调优过程之后，演示了如何比较未优化和优化后的模型的性能。

这里介绍了使用 ResNet-50 v2 本地的简单例子。然而，TVMC 支持更多的功能，包括交叉编译、远程执行和剖析/基准测试（profiling/benchmarking）。

要想知道还有哪些可用的选项，请看 ``tvmc --help``。

在 [用 Python 接口编译和优化模型](auto_tuning_with_pyton) 教程中，将使用 Python 接口介绍同样的编译和优化步骤。